# Move to statkit

In [1]:
def mean_confidence(prob, mean, n, s=None, disp=None, std=None, gen_std=None):
    if not gen_std:
        trust = tinv(prob, n)
        if s:
            std = s #????
        if std:
            std = np.sqrt(n/(n-1)) * std
        if disp:
            std = np.sqrt(n/(n-1)*disp)
    else:
        trust = Laplace_table_inverse(prob)
        std = gen_std

    precision = (trust * std) / np.sqrt(n)
    left, right = map(stat_round, (mean-precision, mean+precision))
    return left, right

In [2]:
def disp_confidence(prob, n, s=None, disp=None, std=None):
    if not s:
        if std:
            s = np.sqrt(n/(n-1))*std
        elif disp:
            s = np.sqrt(n/(n-1) * disp)
        
    alpha_left, alpha_right, k = (1-prob)/2, (1+prob)/2, n-1
    border = lambda alpha: ((n-1) * s**2) / chiinv(alpha, n)
    borders = (border(a) for a in (alpha_left, alpha_right))
    
    left, right = map(stat_round, map(np.sqrt, borders))
    
    return left, right

# Working

In [3]:
from statkit import (seriesBuilder as SB, stattools)
from statkit.functions import *

import numpy as np

In [4]:
data = (263, 284, 310, 296, 288, 251)

In [5]:
series = SB.discreetVariationSeries(data)

In [6]:
series.table

,x,n,w,w_cum
0,251.0,1.0,0.17,0.17
1,263.0,1.0,0.17,0.33
2,284.0,1.0,0.17,0.50
3,288.0,1.0,0.17,0.67
4,296.0,1.0,0.17,0.83
5,310.0,1.0,0.17,1.00
Total,1692.0,6.0,1.00,3.50


In [7]:
stats = stattools.Stats(series)

In [8]:
stats.show_stats()

,mean,mode,median,range,abs_dev,disp,UB disp,std,UB_std,coeff var
0,282.0,251.0,284.0,59.0,16.67,390.33,468.4,19.76,21.64,7.01
